In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import datetime
from IPython.display import display, Markdown

%matplotlib inline

In [ ]:
def timestamper(timestamp):
    return datetime.datetime.fromtimestamp(float(timestamp)/1000)

In [ ]:
files = glob.glob("anomaly_detection/data/raw/*")

In [ ]:
labels = []

try:
    processed = pd.read_csv("processed.csv",index_col=0)
except:
    processed = pd.DataFrame(columns=["file"])
count = 0
for file in files:
    if file not in processed.file:
        df = pd.read_csv(file,index_col="tiempo", parse_dates=True, date_parser=timestamper)
        df_fft = pd.read_csv(file.replace(".csv",".fft.csv").replace("raw","fft"),index_col=0)
        if df_fft.mean().abs().mean() > 1:
            print("filename: {}  {}/{}".format(file,count,len(files)))
            fig,ax = plt.subplots(1,2, figsize=(20,5))
            df.plot(ax=ax[0], title="Time")
            df_fft.plot(ax=ax[1], title="Fourier")
            ax[0].set_ylim((-3,3))
            ax[1].set_ylim((-300,300))

            plt.legend()
            plt.show()
    
            stats = {}
            stats["Time-mean"] = df.mean()
            stats["Time-std"] = df.std()
            stats["Fourier-mean"] = df_fft.iloc[1:].mean()
            stats["Fourier-std"] = df_fft.iloc[1:].std()
            stats = pd.DataFrame(stats).transpose()
            stats["Mean"] = stats.mean(axis=1)
            display(stats)

            label = input("0: Off, 1:Normal, 2:Anomaly ")
            while label not in ["0","1","2"]:
                display(Markdown("**Incorrect Input**"))
                label = input("0: Off, 1:Normal, 2:Anomaly ")
            label = int(label)
        else:
            label = 0
        count +=1
        labels.append(label)
        processed = processed.append({"file":file},ignore_index=True)
processed.to_csv("processed.csv")